In [1]:
from flask import Flask, request, jsonify, redirect, url_for
import numpy as np,pandas as pd
import time,requests
import os,re
from dateutil.relativedelta import relativedelta
import datetime,random
import pymysql as m

import json,ast# ast包可以将字符串转换为有效字典
import hashlib,base64,hmac

import sys
sys.path.append('..')
import Data_Validation_Functions as f

# V 0.2

In [4]:
def webhook(groupid):
    conn = m.connect(host = '172.16.92.96',port=3306,user = 'yonghongbi',password='yonghongbi123',database = 'BIDB')
    name = ['钉钉ID','姓名','职级','部门编号','是否运营系','是否全权限']
    sql = """select webhook from T_Robot_Dim_Group where groupid = '%s' """%(groupid)
    cur = conn.cursor()
    cur.execute(sql)
    return cur.fetchall()[0][0]

def dept_recognize(content):
    if content.find( '营' ) >= 0:
        if content.find( '湖' ) >= 0:
            index_start = content.find( '湖' )
        elif content.find( 'D' ) >= 0:
            index_start = content.find( 'D' )
        elif content.find( '体' ) >= 0:
            index_start = content.find( '体' )
        elif content.find( '商' ) >= 0:
            index_start = content.find( '商' )  

        index_end = content.find( '营' )+1
        departname = content[index_start: index_end]       #没法满足：'一营'
        ctype=2

    elif content.find('团') >= 0:
        if content.find('湖') >=0:
            index_start = content.find('湖')
        elif content.find('D') >=0:
            index_start = content.find('D')
        elif content.find('体') >=0:
            index_start = content.find('体')
        elif content.find('商') >=0:
            index_start = content.find('商')          

        index_end = content.find('团') + 1
        departname = content[index_start: index_end]
        ctype=4
        
    elif content.find( '部' ) >= 0:
        if content.find( '销' ) >= 0:
            index_start = content.find( '销' )
        elif content.find('D') >= 0:
            index_start = content.find( 'D' )
        elif content.find( '体' ) >= 0:
            index_start = content.find( '体' )
        elif content.find( '商' ) >= 0:
            index_start = content.find( '商' )
        
        nums = 0
        while content.find('部',nums)>0:
            index_end = content.find('部',nums) #record
            nums = content.find('部',nums)+1   #穷尽,每次循环需要加1
            
        departname = content[index_start: index_end+1]       #没法满足：'二部'
        ctype=3
        
    elif content.find('组') >= 0:
        try:
            index_start = content.find('组') - 4
        except:
            index_start = content.find('组') - 3
            
        index_end = content.find('组')+1
        departname = content[index_start: index_end] 
        ctype=5
        
    else:
        departname = check_organid(organid)
        
    return departname,ctype

def check_organid(dept):
    conn = m.connect(host = '172.16.92.96',port=3306,user = 'yonghongbi',password='yonghongbi123',database = 'BIDB')
    if dept[0]=='0':
        sql = """select departname from T_Robot_Dim_department where organid ='%s' """%(dept)
    else:
        sql = """select organid from T_Robot_Dim_department where departname = '%s' """%(dept)

    cur = conn.cursor()
    cur.execute(sql)
    result = cur.fetchall()[0][0]

    if result.find('营')>=0:
        ctype = 2
    elif result.find('部')>=0:
        ctype = 3
    elif result.find('团')>=0:
        ctype = 4
    elif result.find('组')>=0:
        ctype = 5
    else:
        ctype = -1
    return result,ctype

def date_granularity(content):
    if content.find('天')>=0 or content.find('日')>=0:
        return '天'
    elif content.find('月') >= 0:
        return '月'        
    elif content.find('年') >= 0:
        return '年'

def date_skip(content):
    if content.find('今')>=0 or content.find('本')>=0 or content.find('这')>=0 :
        return '今'
    elif content.find('昨') >= 0 or content.find('上') >= 0 or content.find('前') >= 0:
        return '上'            
    
def date_sql(require):
    if require['时间粒度'] == '年':
        if require['时间跨度'] == '今':
            date = 'substring(stadate,1,4) =\''+ time.strftime('%Y')+ '\''
        elif require['时间跨度'] == '上':
            date = 'substring(stadate,1,4) =\''+ datetime.datetime.strftime(datetime.datetime.now()+ relativedelta(years=-1) ,"%Y") + '\''

    elif require['时间粒度'] == '月':
        if require['时间跨度'] == '今':
            date = 'substring(stadate,1,7) =\''+ time.strftime('%Y-%m')+ '\''
        elif require['时间跨度'] == '上':
            date = 'substring(stadate,1,7) =\''+ datetime.datetime.strftime(datetime.datetime.now()+ relativedelta(months=-1) ,"%Y-%m")+ '\'' 

    elif require['时间粒度'] == '天' :
        if require['时间跨度'] == '今':
            date = 'substring(stadate,1,10)=\''+ time.strftime('%Y-%m-%d')+ '\''
        if require['时间跨度'] == '上':
            date = 'substring(stadate,1,10)=\''+ datetime.datetime.strftime(datetime.datetime.now()+ relativedelta(days=-1) ,"%Y-%m-%d") + '\''
    return date

In [ ]:
app = Flask(__name__)
h = f.Handler()

@app.route('/', methods=['POST'])
############################################################################## 业绩统计数据
def performance():
    body_data = request.form.to_dict()
    content,organid,groupid = request.form.to_dict().values()
    #organid 是用户所在架构，若需求中无明显的部门架构，则根据他所在的部门查询
    
    robot = f.Robot(test=True)
    robot.webhook = webhook(body_data['groupid'])
    robot.secret = ''
    robot.url = robot.get_url()
    
    print(content)

########################################################### 解析需求
    require={}
#1.计算查询的部门：如果没有详细的部门，就根据问的人的所属架构 
    try:
        if content.find('营')>=0 or content.find('部')>=0 or content.find('团')>=0 or content.find('组')>=0:
            require['departname'],require['ctype'] = dept_recognize(content)   #函数见下面
            require['organid'] = check_organid(require['departname'])[0]
        else:
            require['departname'],require['ctype'] = check_organid(organid)   #函数见下面
            require['organid'] = organid

        depart_sql = {2:'depart_1st', 3:'depart_2nd', 4:'depart_3rd'}
        departid_sql = {2:'departid_1st', 3:'departid_2nd', 4:'departid_3rd'}
        depart = depart_sql[require['ctype']]
        departid = departid_sql[require['ctype']] + ' like \'' + require['organid']+'%\''
    except:
        robot.send_msg(h.dept_e())
        return ''

#2.时间粒度 和 时间跨度
    try:
        require['时间粒度'] = date_granularity(content)   #函数见下面
        require['时间跨度'] = date_skip(content)
        print(require)

#3.根据时间粒度 和 跨度，计算时间条件
        date = date_sql(require)
        print(date)
    except:
        robot.send_msg(h.date_e())
    return ''
########################################################### 解析指标   
    index_pfm = {'下单业绩':'LSF0001','签收业绩':'LSF0002'}

########################################################### 拼接sql 
    try:
        index_value = []
        for i in index_pfm:
            sql_head = ''' select %s , %s ,indcode,indname,sum(numerator_value),sum(denominator_value),sum(indresult)
                     '''%(date[:23],depart)   #(日期粒度，部门粒度) 
                     
            sql_where = ''' from dcp_dw_rpt.ind_result_day
        where ctype in ( %d ) and indcode ='%s'  and %s and %s and type in (toInt64(1),toInt64(2))
        '''%(require['ctype'], index_pfm[i], departid, date) 
    # 重点:        (部门粒度,    指标编号,   部门粒度,  日期)      

            sql_groupby = 'group by %s , %s ,indcode,indname'%(date[:23],depart) 
                                                              #(日期粒度，部门粒度)
            sql = sql_head + sql_where + sql_groupby

            value = str(int(f.query(sql)[0][6]))
            index_value.append( value )
    except:
        print('查询失败，请检查SQL 和 数据库是否有数据')
        robot.send_msg(h.sql_query_e_a() + date[25:-1] + h.sql_query_e_b())
        return ''
########################################################### 拼接结果并发送
    emoji_fun =['[抱拳]','[送花花]','[赞]','[666]','[加油干]','[爱意]','[捧脸]','[收到]','[恭喜]',
'[承让]','[吃瓜]','[耶]','[举手]','[干杯]','[鲜花]','[鸡腿]']
    num2 = random.randint(0, len(emoji_fun)-1)
    emoji = emoji_fun[num2] + ' '
    msg = emoji + require['departname']+' '+ date[25:-1]+'的下单业绩为'+index_value[0]+'，签收业绩'+index_value[1]

    robot.send_msg(msg)
    return ''

if __name__ == '__main__':
    app.run( host='0.0.0.0',port='5001' )

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://0.0.0.0:5001/ (Press CTRL+C to quit)


# 模糊问题V 0.1

In [5]:
app = Flask(__name__)# Flask构造函数使用当前模块（__name __）的名称作为参数。

@app.route('/performance', methods=['POST'])
############################################################################## 业绩统计数据
def performance():
    require = request.form.to_dict()
    print(require)


    if require['时间粒度'] == '年':
        if require['时间'] == '今':
            date = 'and substring(toString(f.order_date),1,4)=\''+ time.strftime('%Y')+ '\''
        elif require['时间'] == '上':
            date = 'and substring(toString(f.order_date),1,4)=\''+ datetime.datetime.strftime(datetime.datetime.now()+ relativedelta(years=-1) ,"%Y") + '\''

    elif require['时间粒度'] == '月':
        if require['时间'] == '今':
            date = 'and substring(toString(f.order_date),1,7)=\''+ time.strftime('%Y-%m')+ '\''
        elif require['时间'] == '上':
            date = 'and substring(toString(f.order_date),1,7)=\''+ datetime.datetime.strftime(datetime.datetime.now()+ relativedelta(months=-1) ,"%Y-%m")+ '\'' 

    elif require['时间粒度'] == '天' :
        if require['时间'] == '今':
            date = 'and substring(toString(f.order_date),1,10)=\''+ time.strftime('%Y-%m-%d')+ '\''
        if require['时间'] == '上':
            date = 'and substring(toString(f.order_date),1,10)=\''+ datetime.datetime.strftime(datetime.datetime.now()+ relativedelta(days=-1) ,"%Y-%m-%d") + '\''
    else:
        date = ''
        
    
    index_pfm = index['下单业绩']+','+index['签收业绩']
    #查询、推送
    sql = '''
select %s
from dw_dmp.dm_fact_performance f
    all inner join dcp_dw_dmp.dim_department_hierarchy d on d.organid = f.share_orgid
    all left join dcp_dw_dmp.dim_staff_info s on s.staffid = f.share_staff and s.company = toInt64(1)
where f.company=toInt64(1) and f.is_achv=toInt64(1) and f.is_shuadan=toInt64(0) and f.share_cash>0 and f.model=toInt64(1)
and f.share_orgid like '%s' %s
'''%(index_pfm,'0261%', date)

    msg = '体管六营 下单业绩'+'，签收业绩'.join([str(int(i)) for i in f.query(sql)[0]])
    robot_answer.send_msg(msg)
    return ''

@app.route('/distribute', methods=['POST'])
############################################################################## 派发
def distribute():
    require = request.form.to_dict()
    print(require)

    if require['时间粒度'] == '年':
        if require['时间'] == '今':
            date = 'and substring(f.dist_date,1,4)=\''+ time.strftime('%Y')+ '\''
        elif require['时间'] == '上':
            date = 'and substring(f.dist_date,1,4)=\''+ datetime.datetime.strftime(datetime.datetime.now()+ relativedelta(years=-1) ,"%Y") + '\''
    elif require['时间粒度'] == '月':
        if require['时间'] == '今':
            date = 'and substring(f.dist_date,1,7)=\''+ time.strftime('%Y-%m')+ '\''
        elif require['时间'] == '上':
            date = 'and substring(f.dist_date,1,7)=\''+ datetime.datetime.strftime(datetime.datetime.now()+ relativedelta(months=-1) ,"%Y-%m")+ '\'' 

    elif require['时间粒度'] == '天' :
        if require['时间'] == '今':
            date = 'and substring(f.dist_date,1,10)=\''+ time.strftime('%Y-%m-%d')+ '\''
        if require['时间'] == '上':
            date = 'and substring(f.dist_date,1,10)=\''+ datetime.datetime.strftime(datetime.datetime.now()+ relativedelta(days=-1) ,"%Y-%m-%d") + '\''
    else:
        date = ''
        
    index_dist = index['派发新客数']+','+index['下单新客数']+','+index['下单新客成交率']
    sql = '''select %s
    from dcp_dw_dmp.dw_fact_distribute f
        all inner join dcp_dw_dmp.dim_department_hierarchy d on d.organid = f.dist_orgid
        all left join dcp_dw_dmp.dim_staff_info s on s.staffid = f.salesstaff and s.company = toInt64(1)
    where f.company='1' and f.is_recycle='0'
    and f.dist_orgid like '%s'  %s
    '''%( index_dist, '0261%', date)

    index_name = ['派发新客数','，下单新客数','，下单新客成交率']
    result = f.query(sql)[0]
    index_value = [str(int(result[0])),str(int(result[1])),str(round(float(a[2]*100),2))]

    empty=''
    for i in range(3):
        empty = empty+ index_name[i] + index_value[i]

    msg = '体管六营 '+ empty + '%'
    robot_answer.send_msg(msg)
    return ''

@app.route('/fans', methods=['POST'])
############################################################################## 加粉
def fans():
    require = request.form.to_dict()
    print(require)

    if require['时间粒度'] == '年':
        if require['时间'] == '今':
            date = 'and substring(f.stat_date,1,4)=\''+ time.strftime('%Y')+ '\''
        elif require['时间'] == '上':
            date = 'and substring(f.stat_date,1,4)=\''+ datetime.datetime.strftime(datetime.datetime.now()+ relativedelta(years=-1) ,"%Y") + '\''
    elif require['时间粒度'] == '月':
        if require['时间'] == '今':
            date = 'and substring(f.stat_date,1,7)=\''+ time.strftime('%Y-%m')+ '\''
        elif require['时间'] == '上':
            date = 'and substring(f.stat_date,1,7)=\''+ datetime.datetime.strftime(datetime.datetime.now()+ relativedelta(months=-1) ,"%Y-%m")+ '\'' 

    elif require['时间粒度'] == '天' :
        if require['时间'] == '今':
            date = 'and substring(f.stat_date,1,10)=\''+ time.strftime('%Y-%m-%d')+ '\''
        if require['时间'] == '上':
            date = 'and substring(f.stat_date,1,10)=\''+ datetime.datetime.strftime(datetime.datetime.now()+ relativedelta(days=-1) ,"%Y-%m-%d") + '\''
    else:
        date = ''
        
    index_fans = index['总进粉']+','+ index['下单客户数']+','+index['下单总进粉成交率']

    sql = '''select %s
    from dcp_dw_rpt.t_rpt_fans_performance_lvshou_mid f
    where f.wx_channel_dep ='绿瘦商城' and f.wx_company='微雅官方' and f.channel_2nd <>'丽茜科普'
    and f.organid like '%s' %s
    '''%( index_fans,'025901%', date)

    index_name = ['总进粉','，下单客户数','，下单总进粉成交率']
    result = f.query(sql)[0]
    index_value = [str(int(result[0])),str(int(result[1])),str(round(float(a[2]*100),2))]

    empty=''
    for i in range(3):
        empty = empty+ index_name[i] + index_value[i]

    msg = '商城 '+ empty + '%'
    robot_answer.send_msg(msg)
    return ''

if __name__ == '__main__':
    app.run(host='0.0.0.0',port='28083')

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://0.0.0.0:28083/ (Press CTRL+C to quit)
